In [2]:
import os, time
from datetime import datetime
from typing import Optional
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import DeepResearchTool, MessageRole, ThreadMessage
from dotenv import load_dotenv
load_dotenv()

def fetch_and_print_new_agent_response(
    thread_id: str,
    agents_client: AgentsClient,
    last_message_id: Optional[str] = None,
) -> Optional[str]:
    lastest_agent_message = agents_client.messages.get_last_message_by_role(
        thread_id=thread_id,
        role=MessageRole.AGENT,
    )
    if not lastest_agent_message or lastest_agent_message.id == last_message_id:
        print(f"😶 No new content. last_message_id: {last_message_id}")
        return last_message_id  # No new content

    print(f"\n===================== 🤖 Agent response with lastest_agent_message id: {lastest_agent_message.id} ===========================")
    print("\n".join(t.text.value for t in lastest_agent_message.text_messages))

    for ann in lastest_agent_message.url_citation_annotations:
        print(f"****** 🛜 URL Citation ******:\n   [{ann.url_citation.title}]({ann.url_citation.url})")

    return lastest_agent_message.id


def create_research_summary(
        message : ThreadMessage,
        base_filename: str = "research_summary"
) -> None:
    if not message:
        print("No message content provided, cannot create research summary.")
        return

    # Generate filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = f"{base_filename}_{timestamp}.md"

    with open(filepath, "w", encoding="utf-8") as fp:
        # Write text summary
        text_summary = "\n\n".join([t.text.value.strip() for t in message.text_messages])
        fp.write(text_summary)
        # if text_summary is less than 1000 characters, print to the console
        if len(text_summary) < 1000:
            print(f"🔍🔍🔍 Research summary: {text_summary}")

        # Write unique URL citations, if present
        if message.url_citation_annotations:
            fp.write("\n\n## References\n")
            seen_urls = set()
            for ann in message.url_citation_annotations:
                url = ann.url_citation.url
                title = ann.url_citation.title or url
                if url not in seen_urls:
                    fp.write(f"- [{title}]({url})\n")
                    seen_urls.add(url)

    print(f"Research summary written to '{filepath}'.")


def process_agent_run(agents_client: AgentsClient, thread_id: str, agent_id: str, user_message: str) -> None:
    """Process a single agent run with the given user message"""
    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread_id,
        role="user",
        content=user_message,
    )
    print(f"Created message, ID: {message.id}")
    print(f"👤 User message: {user_message}")

    print(f"Start processing the message... this may take a few minutes to finish. Be patient!")
    # Poll the run as long as run status is queued or in progress
    run = agents_client.runs.create(thread_id=thread_id, agent_id=agent_id)
    last_message_id = None
    while run.status in ("queued", "in_progress"):
        time.sleep(2)
        run = agents_client.runs.get(thread_id=thread_id, run_id=run.id)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        print(f"\n\n🔄🔄🔄 Run status: {run.status} at {timestamp}")

        last_message_id = fetch_and_print_new_agent_response(
            thread_id=thread_id,
            agents_client=agents_client,
            last_message_id=last_message_id,
        )

    print(f"\n\n🎉🎉🎉 Run finished with status: {run.status}, ID: {run.id}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
        return

    # Fetch the final message from the agent in the thread and create a research summary
    final_message = agents_client.messages.get_last_message_by_role(
        thread_id=thread_id, role=MessageRole.AGENT
    )
    if final_message:
        create_research_summary(final_message)


project_client = AIProjectClient(
    endpoint=os.environ["DEEP_RESEARCH_PROJECT_CONNECTION_STRING"],
    credential=DefaultAzureCredential(),
)

conn_id = project_client.connections.get(name=os.environ["DEEP_RESEARCH_BING_RESOURCE_NAME"]).id


# Initialize a Deep Research tool with Bing Connection ID and Deep Research model deployment name
deep_research_tool = DeepResearchTool(
    bing_grounding_connection_id=conn_id,
    deep_research_model=os.environ["DEEP_RESEARCH_MODEL_DEPLOYMENT_NAME"],
)

# Create Agent with the Deep Research tool and process Agent run
with project_client:

    with project_client.agents as agents_client:

        # Create a new agent that has the Deep Research tool attached.
        # NOTE: To add Deep Research to an existing agent, fetch it with `get_agent(agent_id)` and then,
        # update the agent with the Deep Research tool.
        agent = agents_client.create_agent(
            model=os.environ["DEEP_RESEARCH_CHAT_MODEL_DEPLOYMENT_NAME"],
            name="deep-research-agent",
            instructions="You are a helpful Agent that assists in researching scientific topics.",
            tools=deep_research_tool.definitions,
        )

        # [END create_agent_with_deep_research_tool]
        print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = agents_client.threads.create()
        print(f"Created thread, ID: {thread.id}")

        # Multi-turn conversation loop
        # ❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️
        first_message = "微软AI最新进展. You can ask clarification questions if needed."
        current_message = first_message
        
        while True:
            # Process the current message
            process_agent_run(agents_client, thread.id, agent.id, current_message)
            
            # Ask user if they want to continue
            print("\n" + "="*60)
            user_input = input("Do you want to continue the conversation? Enter 'n' to quit, or type your next question: ")
            print(f"********* 💬 User input: {user_input} *********")
            
            # Check if user wants to quit
            if user_input.strip().lower() == 'n':
                print("Ending conversation...")
                break
            
            # Use user input as the next message
            current_message = user_input.strip()
            if not current_message:
                print("Empty message, ending conversation...")
                break

        # Clean-up and delete the agent once the conversation is finished.
        # NOTE: Comment out this line if you plan to reuse the agent later.
        # agents_client.delete_agent(agent.id)
        # print("Deleted agent")

Created agent, ID: asst_v0kWQaQZXgnFujTkucwvpduc
Created thread, ID: thread_BAu5gHD4dzNbbeVoDWGZUpSx
Created message, ID: msg_Zxh7aftQE7TNsSeRelFYAEru
👤 User message: 微软AI最新进展. You can ask clarification questions if needed.
Start processing the message... this may take a few minutes to finish. Be patient!


🔄🔄🔄 Run status: RunStatus.IN_PROGRESS at 20250806_144101

===================== 🤖 Agent response with lastest_agent_message id: msg_semBXTctGqaCg8HoNMWNKDoj ===========================



🔄🔄🔄 Run status: RunStatus.COMPLETED at 20250806_144104
😶 No new content. last_message_id: msg_semBXTctGqaCg8HoNMWNKDoj


🎉🎉🎉 Run finished with status: RunStatus.COMPLETED, ID: run_ZU1OXh7uXIUJ12f5Xe5hERX7
🔍🔍🔍 Research summary: 请问您对“微软AI最新进展”具体有哪些方面感兴趣？我可以根据您的兴趣点更有针对性地进行研究。以下是几个可以展开的方向，您可以选择或补充：  

1. **技术进展**：微软在AI技术（如大语言模型、深度学习、计算机视觉等）方面的最新研究或成果。  
2. **应用场景**：AI技术在微软产品（如Office、Azure、LinkedIn等）中的具体应用有哪些新发展？  
3. **商业合作**：微软在AI领域有哪些新的合作伙伴关系或市场推广计划？  
4. **开源项目**：Azure OpenAI或其他微软支持的开源AI平台的最新更新。  
5

In [ ]:
def print_agent_message_by_id(
    thread_id: str,
    agents_client: AgentsClient,
    message_id: str,
):
    """
    根据 message_id 打印该消息的详细信息。
    """
    message = agents_client.messages.get(
        thread_id=thread_id,
        message_id=message_id,
    )
    if not message:
        print(f"😶 未找到消息，message_id: {message_id}")
        return
    
    print(f"🔍🔍🔍 message: {message}")
    print(f"\n===================== 🤖 Agent message info, id: {message.id} ===========================")
    print("\n".join(t.text.value for t in message.text_messages))

    for ann in getattr(message, "url_citation_annotations", []):
        print(f"****** 🛜 URL Citation ******:\n   [{ann.url_citation.title}]({ann.url_citation.url})")


project_client = AIProjectClient(
    endpoint=os.environ["DEEP_RESEARCH_PROJECT_CONNECTION_STRING"],
    credential=DefaultAzureCredential(),
)


# Create Agent with the Deep Research tool and process Agent run
with project_client:

    with project_client.agents as agents_client:

        thread_id = "thread_BAu5gHD4dzNbbeVoDWGZUpSx"
        message_id = "msg_Zxh7aftQE7TNsSeRelFYAEru"
        print_agent_message_by_id(thread_id, agents_client, message_id)

KeyError: 'PROJECT_ENDPOINT'

In [ ]:
def print_agent_message_by_id(
    thread_id: str,
    agents_client: AgentsClient,
    message_id: str,
):
    """
    根据 message_id 打印该消息的详细信息。
    """
    message = agents_client.messages.get(
        thread_id=thread_id,
        message_id=message_id,
    )
    if not message:
        print(f"😶 未找到消息，message_id: {message_id}")
        return
    
    print(f"🔍🔍🔍 message: {message}")
    print(f"\n===================== 🤖 Agent message info, id: {message.id} ===========================")
    print("\n".join(t.text.value for t in message.text_messages))

    for ann in getattr(message, "url_citation_annotations", []):
        print(f"****** 🛜 URL Citation ******:\n   [{ann.url_citation.title}]({ann.url_citation.url})")


project_client = AIProjectClient(
    endpoint=os.environ["DEEP_RESEARCH_PROJECT_CONNECTION_STRING"],
    credential=DefaultAzureCredential(),
)


# Create Agent with the Deep Research tool and process Agent run
with project_client:

    with project_client.agents as agents_client:

        thread_id = "thread_Dv168AXLirnEa3nDftDz5GvB"
        message_id = "msg_dO4RV00wB8DF9rEE00EJdqwI"
        print_agent_message_by_id(thread_id, agents_client, message_id)

🔍🔍🔍 message: {'id': 'msg_dO4RV00wB8DF9rEE00EJdqwI', 'object': 'thread.message', 'created_at': 1752464586, 'assistant_id': 'asst_u1c21zl3lL0sH2go4uTJlnd7', 'thread_id': 'thread_Dv168AXLirnEa3nDftDz5GvB', 'run_id': 'run_AR5h8wCadc5ZeMPoiuigc093', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Final Report:\n# 微软人工智能产品最新动态综述（2023–2025）\n\n## Azure AI 平台的创新与进展\n\n微软在 2023 年大力发展其 Azure AI 云平台，提供强大的基础模型服务和开发工具，为企业和开发者带来新一代 AI 能力。其中，**Azure OpenAI Service** 于2023年初正式推出商用，整合了 OpenAI 的大型语言模型（包括 GPT-3.5、ChatGPT 等）到 Azure 云上【47:2†source】。2023年3月，微软宣布 OpenAI 最新的 GPT-4 模型在 Azure 上预览上线，使企业可以申请使用这一先进模型构建应用【47:2†source】,【47:2†source】。仅一年内，Azure OpenAI 服务已吸引超过18,000家客户使用生成式 AI【47:1†source】。Azure 在5月的 Build 2023 发布会上进一步宣布 GPT-4 模型全面开放，以及一系列增强功能，用于帮助开发者将生成式 AI 应用于自身数据和应用【47:3†source】,【47:3†source】。\n\nAzure 平台不断扩展模型和功能阵容。2023年下半年，微软在 Ignite 2023 大会上推出多模态和强化版模型预览，例如**GPT-4 Turbo with Vision**（具备视觉输入理解能力）和图像生成模型 **DALL·E 3** 等，帮助用户实现跨文字、图像和视频的内容生成【47:1†source】。同时，新模型 **GPT-4 Turbo** 提